In [1]:
import pandas as pd
import numpy as np
import psycopg2
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import nltk
import sys
from sqlalchemy import create_engine
import string
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split, cross_val_score

In [2]:
engine = create_engine('postgresql://teresaborcuch@localhost:5433/capstone')

In [275]:
query = "SELECT DISTINCT ON(title) title, date, author, body, link, section FROM nytimes;"

In [276]:
data = pd.read_sql(query, engine)

In [277]:
data.head()

,title,date,author,body,link,section
0,$5 Million for a Super Bowl Ad. Another Millio...,20170129,Sapna Maheshwari,"This month, Anheuser-Busch InBev hosted a doze...",http://www.nytimes.com/2017/01/29/business/5-m...,business
1,"1 Patient, 7 Tumors and 100 Billion Cells Equa...",20161207,Denise Grady,The remarkable recovery of a woman with advanc...,http://www.nytimes.com/2016/12/07/health/cance...,health
2,15 of the Best Journals by Our Reporters Aroun...,20161230,Barbara Tierney,Our foreign correspondents wrote about dozens ...,http://www.nytimes.com/2016/12/30/world/15-of-...,world
3,2 Charged in Ponzi Scheme Built Around ‘Hamilt...,20170127,Eli Rosenberg,Two men were accused on Friday of bilking mill...,http://www.nytimes.com/2017/01/27/nyregion/ham...,nyregion
4,2017 Pegasus World Cup Features California Chr...,20170127,Melissa Hoppert,"HALLANDALE BEACH, Fla. — Last January, an emai...",http://www.nytimes.com/2017/01/27/sports/horse...,sports


In [279]:
data.to_csv('/Users/teresaborcuch/Desktop/nyt_backup.csv', encoding = 'utf8')

In [10]:
for i in data['title']:
    i = i.encode('utf-8')

In [11]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [95]:
chars = string.punctuation + '’'

In [105]:
chars

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\xe2\x80\x99'

In [107]:
'’'

'\xe2\x80\x99'

In [24]:
print 0xe2

226


In [32]:
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    text = text.encode('ascii', errors = 'replace')
    text = ''.join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [33]:
c_vec = TfidfVectorizer(decode_error = 'replace', tokenizer = tokenize, strip_accents = 'unicode', stop_words = 'english')
c_vec.fit(data['title'])

TfidfVectorizer(analyzer=u'word', binary=False, decode_error='replace',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x11aa27cf8>, use_idf=True,
        vocabulary=None)

In [34]:
v_titles = pd.DataFrame(c_vec.transform(data['title']).todense(), columns = c_vec.get_feature_names())

In [35]:
v_titles.head()

,1,100,11th,15,16000,2,20,2017,21stcenturi,3,...,young,younger,youv,yuliya,zealand,zealou,zhiqiang,zimmerman,zombi,zoo
0,0.560348,0.280174,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.496692,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.000000,0.0,0.373307,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.315749,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
v_titles.sum(axis=0).sort_values(ascending = False).head(5)

trump     13.355359
ban        4.164394
new        4.016073
order      3.647506
immigr     3.565345
dtype: float64

In [37]:
# make y opinion vs non-opinion
y = [1 if i == 'opinion' else 0 for i in data['section']]

In [38]:
# feature selection
rfecv = RFECV(estimator = DecisionTreeClassifier(), cv = 3, scoring = 'mean_squared_error')
rfecv.fit(v_titles, y)

RFECV(cv=3,
   estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
   estimator_params=None, scoring='mean_squared_error', step=1, verbose=0)

In [39]:
rfecv_cols = v_titles.columns[rfecv.support_]

In [40]:
len(rfecv_cols)

1133

In [41]:
X = v_titles[rfecv_cols]

In [48]:
# make train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [43]:
# fit decision tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
score = np.mean(cross_val_score(dt, X_test, y_test, cv = 5, scoring = "accuracy"))

In [44]:
score

0.85665634674922608

In [141]:
# 80% of the articles are opinion
1- 58.0/len(data)

0.8060200668896321

# Naive Bayes

In [169]:
from nltk.corpus import movie_reviews
import random
documents = [(list(movie_reviews.words(fileid)), category) for category in movie_reviews.categories() for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

In [170]:
documents[0]

([u'please',
  u'don',
  u"'",
  u't',
  u'mind',
  u'this',
  u'windbag',
  u'letting',
  u'off',
  u'a',
  u'bit',
  u'of',
  u'steam',
  u'.',
  u'.',
  u'.',
  u'i',
  u'just',
  u'want',
  u'to',
  u'warn',
  u'all',
  u'of',
  u'y',
  u"'",
  u'all',
  u'not',
  u'to',
  u'waste',
  u'your',
  u'hard',
  u'-',
  u'earned',
  u'$$$',
  u'on',
  u'anaconda',
  u'.',
  u'it',
  u"'",
  u's',
  u'not',
  u'even',
  u'worth',
  u'a',
  u'99',
  u'-',
  u'cent',
  u'video',
  u'rental',
  u'.',
  u'don',
  u"'",
  u't',
  u'listen',
  u'to',
  u'ebert',
  u'on',
  u'this',
  u'one',
  u'(',
  u'he',
  u'has',
  u'no',
  u'clue',
  u'what',
  u'he',
  u"'",
  u's',
  u'talking',
  u'about',
  u'!',
  u')',
  u'btw',
  u'i',
  u'accompanied',
  u'my',
  u'friend',
  u'to',
  u'this',
  u'one',
  u'because',
  u'she',
  u'was',
  u'*',
  u'required',
  u'*',
  u'to',
  u'watch',
  u'it',
  u'on',
  u'assignment',
  u'(',
  u'she',
  u'reviews',
  u'movies',
  u'for',
  u'a',
  u'local',
 

In [171]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())

In [174]:
all_words

FreqDist({u'sonja': 1,
          u'woods': 90,
          u'spiders': 4,
          u'bazooms': 1,
          u'hanging': 59,
          u'francesca': 4,
          u'comically': 12,
          u'localized': 1,
          u'disobeying': 1,
          u'hennings': 2,
          u'canet': 1,
          u'scold': 2,
          u'originality': 58,
          u'caned': 1,
          u'rickman': 15,
          u'slothful': 1,
          u'wracked': 1,
          u'stipulate': 1,
          u'capoeira': 1,
          u'rawhide': 2,
          u'bringing': 81,
          u'four': 282,
          u'liaisons': 9,
          u'grueling': 1,
          u'sommerset': 4,
          u'wooden': 48,
          u'wednesday': 6,
          u'broiled': 1,
          u'circuitry': 3,
          u'tak': 1,
          u'crotch': 11,
          u'elgar': 3,
          u'stereotypical': 47,
          u'busybody': 3,
          u'gavan': 1,
          u'inevitably': 35,
          u'snuggles': 1,
          u'francesco': 2,
          u'feasibili

In [59]:
word_features = list(all_words)[:2000]

In [175]:
word_features

[u'all',
 u'definit',
 u'suburban',
 u'global',
 u'farhad',
 u'lebanon',
 u'four',
 u'protest',
 u'mile',
 u'confer',
 u'publish',
 u'verg',
 u'go',
 u'follow',
 u'votefraud',
 u'livetweet',
 u'children',
 u'literari',
 u'accus',
 u'mutini',
 u'william',
 u'pope',
 u'turkish',
 u'tv',
 u'elsewher',
 u'vs',
 u'young',
 u'iraqi',
 u'activist',
 u'reshap',
 u'under',
 u'terror',
 u'aris',
 u'set',
 u'scrutini',
 u'inject',
 u'religi',
 u'digit',
 u'woman',
 u'street',
 u'string',
 u'pedialyt',
 u'nomine',
 u'rise',
 u'voic',
 u'util',
 u'psychologist',
 u'stolen',
 u'facebook',
 u'should',
 u'disast',
 u'fall',
 u'veri',
 u'affect',
 u'ticket',
 u'fan',
 u'wind',
 u'laura',
 u'school',
 u'heaven',
 u'millenni',
 u'turn',
 u'dictatorship',
 u'impact',
 u'did',
 u'gender',
 u'sundanc',
 u'denounc',
 u'colleagu',
 u'democrat',
 u'pea',
 u'solid',
 u'financ',
 u'race',
 u'team',
 u'buddha',
 u'barbara',
 u'heartland',
 u'pleas',
 u'revolution',
 u'iran',
 u'ten',
 u'protect',
 u'battl',
 u'an

In [60]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [162]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [164]:
train_set[0]

({'contains(expand)': False,
  'contains(microsoft)': False,
  'contains(chrome)': False,
  'contains(privat)': False,
  'contains(coverag)': False,
  'contains(cancel)': False,
  'contains(cancer)': False,
  'contains(bogg)': False,
  'contains(veri)': False,
  'contains(million)': False,
  'contains(illeg)': False,
  'contains(that)': False,
  'contains(syrian)': False,
  'contains(trout)': False,
  'contains(pave)': False,
  'contains(confid)': False,
  'contains(secretari)': False,
  'contains(intrus)': False,
  'contains(gag)': False,
  'contains(short)': False,
  'contains(gift)': False,
  'contains(painless)': False,
  'contains(from)': False,
  'contains(smog)': False,
  'contains(colleg)': False,
  'contains(understand)': False,
  'contains(play)': False,
  'contains(strength)': False,
  'contains(farm)': False,
  'contains(percent)': False,
  'contains(destroy)': False,
  'contains(social)': False,
  'contains(bhutan)': False,
  'contains(bike)': False,
  'contains(case)': Fa

## Preprocessing titles for NLTK Naive Bayes

In [257]:
# make these functions part of the list of words for documents
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        if stemmer.stem(item) not in nltk.corpus.stopwords.words('english'):
            stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    text = text.encode('ascii', errors = 'replace')
    text = ''.join([ch.lower() for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [260]:
# preprocessing for NLTK Naive Bayes
# titles have been tokenized, accents/punctuation/stopwords removed
#documents = [(tokenize(title), label) for title in data['title'] for label in y]
labels = ["opinion" if i == 'opinion' else "not opinion" for i in data['section']]
tokenized_titles = [tokenize(title) for title in data['title']]
documents = zip(tokenized_titles, labels)

In [261]:
# make a giant list of all the words in the titles
word_list = []
for title in data['title']:
    word_list.extend(tokenize(title))
all_words = nltk.FreqDist(w.lower() for w in word_list)

In [263]:
all_words.most_common(5)

[(u'trump', 65), (u'new', 15), (u'ban', 13), (u'may', 11), (u'order', 11)]

In [264]:
word_features = list(all_words)[:2000]

In [265]:
def doc_features(doc):
    doc_words = set(doc)
    features = {}
    for word in word_features:
        features[word] = (word in doc_words)
    return features

In [266]:
feature_sets = [(doc_features(d), c) for (d,c) in documents]

In [267]:
len(feature_sets)

299

In [268]:
# split into train and tests
train_set, test_set = feature_sets[90:], feature_sets[:90]

In [269]:
len(train_set)

209

In [270]:
# train the classifier
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [271]:
# test it on test set
print(nltk.classify.accuracy(classifier, test_set))

0.422222222222


In [272]:
# get most informative features
classifier.show_most_informative_features(50)

Most Informative Features
                    vote = True           opinio : not op =      6.5 : 1.0
                    mari = True           opinio : not op =      6.5 : 1.0
                     whi = True           opinio : not op =      6.5 : 1.0
                   speak = True           opinio : not op =      6.5 : 1.0
                   white = True           opinio : not op =      3.9 : 1.0
                 assault = True           opinio : not op =      3.9 : 1.0
                  dakota = True           opinio : not op =      3.9 : 1.0
                  repeal = True           opinio : not op =      3.9 : 1.0
                    park = True           opinio : not op =      3.9 : 1.0
                   south = True           opinio : not op =      3.9 : 1.0
                    rule = True           opinio : not op =      3.9 : 1.0
                      mr = True           opinio : not op =      3.9 : 1.0
                  lawmak = True           opinio : not op =      3.9 : 1.0